<a href="https://www.kaggle.com/code/nazlias/unet-crack-seg?scriptVersionId=94784097" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from glob import glob
from PIL import Image
import random


**collect()**


Dataset dataforcrack adlı dosyadaki train ve test dosyalarından elde edilir. Bu dosyalarda images ve masks olmak üzere iki ayrı dosya bulunur.
Collect fonksiyonu path1 ve path2'de belirtilen dosyalardaki resimleri ve her bir dosyadan alınacak resim sayısını parametre olarak alır. 


Train ve test dosyaları için fonksiyon iki kez çalışır.
Images ve masks dosyalarında bulunan resimler alınarak iki ayrı for döngüsünde imread, resize, cast ve append metodları uygulanarak resimler uygun hale getirilir. Sonrasında image_list ve mask_list olmak üzere iki ayrı listeye kaydedilir.


In [ ]:
def collect(path,low, high):
    
    image_list = []
    mask_list = []

    a = low
    images=sorted(os.listdir(path + '/images/'))
    
    for filename in images:
        a += 1
        
        image = cv2.imread(path + '/images/'+ filename, cv2.IMREAD_COLOR)
        
   
        image = cv2.resize(image, (128, 128),interpolation=cv2.INTER_AREA)
      
        image = tf.cast(image, tf.float32) / 255.0

        image_list.append(image)
    
   
        if a == high:
            break

    a = low
    masks=sorted(os.listdir(path + '/masks/'))
    for filename in masks:
        a += 1
        mask = cv2.imread(path + '/masks/' +filename,cv2.IMREAD_GRAYSCALE)
        mask = cv2.resize(mask, (128, 128))
        mask = np.expand_dims(mask, axis=-1)
        mask = tf.cast(mask, tf.float32) / 255.0
        mask_list.append(mask)

        if a == high:
            break




    return image_list, mask_list

**tf_dataset()**

Çeşitli işlemler uygulandıktan sonra her bir dosya için(train, test) iki ayrı liste bulunur. 
Bu listeler tf_dataset() fonksiyonu içerisinde iki ayrı datasete dönüştürülür.(train_dataset, test_dataset)

In [ ]:
from keras.preprocessing.image import save_img
path1="../input/dataforcrack/crack_segmentation_dataset/train"
path2="../input/dataforcrack/crack_segmentation_dataset/test"
x,y=collect(path1,0,9600)
z,t=collect(path2,0,1600)
"""
x=tf.keras.utils.save_img(
    x=x, data_format=None, file_format=None, scale=True,
)
"""


def tf_dataset(X, Y, batch=64):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))

    dataset = dataset.batch(batch,drop_remainder=True)

    dataset = dataset.prefetch(10)

    return dataset 

train_dataset = tf_dataset(x,y)

test_dataset = tf_dataset(z,t)
print(train_dataset)
print(test_dataset)

In [ ]:
plt.imshow(x[7])
print(plt.show)

In [ ]:
plt.imshow(y[7])
print(y[7][64])

**double_conv_block()**

Bu fonksiyon içerisinde U-Net modeli için gereken encode ve decode kısımları oluşturulur. 
downsample_block() fonksiyonu ile encode, upsample_block() fonksiyonu ile decode oluşturulurken double_conv_block hem encode hem decode kısmında kullanıldığı için ayrı bir fonksiyon olarak yazılmıştır. Aşağıda fonksiyonda verilen parametreler açıklanmıştır.

n_filters-> filtre sayısı

kernel_initalizer->Çekirdek ağırlıkları matrisi için başlatıcıdır.

padding-> girdi ve çıktı resmin aynı boyutta olmasını sağlar.

activation-> ağırlıklı toplamı hesaplayarak ve buna yanlılık ekleyerek bir nöronun etkinleştirilip etkinleştirilmemesi gerektiğine karar verir.

In [ ]:
def double_conv_block(x,n_filters):
    
    x=layers.Conv2D(n_filters,3,padding="same",activation="relu",kernel_initializer="he_normal")(x)
    x=layers.Conv2D(n_filters,3,padding="same",activation="relu",kernel_initializer="he_normal")(x)
    return x
     

def downsample_block(x,n_filters):
    f=double_conv_block(x,n_filters)
    p=layers.MaxPool2D(2)(f)
    p=layers.Dropout(0.3)(p)
    return f,p


def upsample_block(x,conv_features,n_filters):
    
    
    
    x=layers.Conv2DTranspose(n_filters,3,2,padding="same")(x)
    x=layers.concatenate([x,conv_features])
    x=layers.Dropout(0.3)(x)
    x=double_conv_block(x,n_filters)
    return x

**build_unet_model()**

Yukaridaki fonksiyonları kullanarak istenilen U-Net Modelinin oluşturulduğu fonksiyondur. Aşağıda fonksiyonda verilen parametreler açıklanmıştır.

f1, f2, f3, f4->Convolution katmanı uygulanmıştır.

p1,p2,p3,p4->MaxPool2D fonksiyonu ile resmin boyutları ikiye bölünmüştür.

bottleneck->Katmanlarda encode işlemi bittikten ve decode işlemi başlamadan önce oluşan dar boğazdır.

u6, u7, u8, u9->Decode edilen katmanlardır. Bu katmanlara bir önceki decode edilmiş katman ve o sırada denk olan ve daha önceden encode edilmiş (f1,f2,f3,f4) olan katmanlar gönderilerek iki kısmın birleştirilmesi ve sonraki katmanı oluşturması sağlanır.

3->Filtre sayısı


1->Convolution katmanının yüksekliği ve genişliği

outputs->Encode ve decode edildikten sonra oluşan katmandır.

checkpoint ->Model ağırlıklarının kaydedileceği dizindir.

cp_callback->Her 5 epoch'da 1 modelin ağırlıklarını kaydeder.


NUM_EPOCHS->Öğrenme algoritmasının tüm eğitim veri kümesi boyunca çalışacağı sayıyı tanımlayan bir hiperparametredir.

In [ ]:
def build_unet_model():

    inputs=layers.Input(shape=(128,128,3))
  # encoder: contracting path - downsample
    f1,p1=downsample_block(inputs,64)
    f2,p2=downsample_block(p1,128)
    f3,p3=downsample_block(p2,256)
    f4,p4=downsample_block(p3,512)

  

    bottleneck=double_conv_block(p4,1024)

  # decoder: expanding path - upsample
    u6=upsample_block(bottleneck,f4,512)
    u7=upsample_block(u6,f3,256)
    u8=upsample_block(u7,f2,128)
    u9=upsample_block(u8,f1,64)

    outputs=layers.Conv2D(1,1,padding="same",activation = "sigmoid")(u9)
    print(outputs.shape)

    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")
    return unet_model


BUFFER_SIZE = 100

checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
#Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*200)

unet_model = build_unet_model()
# Save the weights using the `checkpoint_path` format
unet_model.save_weights(checkpoint_path.format(epoch=0))



unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="binary_crossentropy",
                  metrics=['accuracy'] )
                
NUM_EPOCHS = 200
TRAIN_LENGTH = 9600

VAL_SUBSPLITS = 5

TEST_LENGTH = 1600


model_history = unet_model.fit(train_dataset,
                              epochs=NUM_EPOCHS,
                              callbacks=[cp_callback],
                              validation_data=test_dataset)

For döngüsü ile eğitilen model üzerinden 100 ve 105 arasında bulunan resimlere maske uygulanmış ve çıktılar alınmıştır.

In [ ]:
for a in range(100,105):
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.axis(False)
    plt.title("Original Image", fontweight="bold")
    
    plt.imshow(x[a])

    plt.subplot(1,2,2)
    plt.axis(False)
    plt.title("Segmentation Mask", fontweight="bold")
    img = np.expand_dims(x[a],axis=0)
    last = unet_model.predict(img)[0,:,:,:]
    plt.imshow(last * 255.0)